Load:

In [1]:
import os, pandas as pd, numpy as np, entsoe
from entsoe import EntsoePandasClient
from entsoe.exceptions import NoMatchingDataError
entsoeToken = os.environ.get('ENTSOETOKEN') # stored token as an environmental variable
client = EntsoePandasClient(api_key=entsoeToken)

Start and end dates:

In [2]:
settings = {}
settings['tz'] = 'Europe/Brussels'
settings['tStart'] = {'year': '2023', 'month': '01', 'day': '01'}
settings['tEnd'] = {'year': '2023', 'month': '06', 'day': '30'}
start = pd.Timestamp(''.join(settings['tStart'].values()), tz='Europe/Brussels')
end = pd.Timestamp(''.join(settings['tEnd'].values()), tz='Europe/Brussels')

List of all bidding areas to loop over:

In [3]:
areasAll = entsoe.mappings.Area._member_names_.copy()

Get day ahead prices by looping over bidding areas; if no data exists for the specific area (NoMatchingData) - remove this key and continue:

In [4]:
dayAheadPrices = dict.fromkeys(areasAll)

Loop through:

In [5]:
iterThrough = [k for k,v in dayAheadPrices.items() if v is None]
for k in iterThrough:
    try:
        dayAheadPrices[k] = client.query_day_ahead_prices(k, start = start, end = end)
    except NoMatchingDataError:
        dayAheadPrices.pop(k)

Connection Error, retrying in 0 seconds


ConnectionError: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))

### Unpacking the method ```query_day_ahead_prices``` 

(following https://github.com/EnergieID/entsoe-py/blob/master/entsoe/entsoe.py)

In [8]:
from entsoe.entsoe import *
URL = 'https://web-api.tp.entsoe.eu/api'
base_params = {'securityToken': client.api_key,
               'periodStart': client._datetime_to_str(start),
               'periodEnd': client._datetime_to_str(end)
              }
area = lookup_area('DK_1') # specific area to look for
area.code

'10YDK-1--------W'

These params are established in ```query_day_ahead_prices``` method:

In [9]:
params = {'documentType': 'A44',
          'in_Domain': area.code,
          'out_Domain': area.code
         }
params.update(base_params)

Query:

In [10]:
testQuery = client.session.get(url = URL, params = params)

The URL passed to restful API:

In [12]:
testQuery.url

'https://web-api.tp.entsoe.eu/api?documentType=A44&in_Domain=10YDK-1--------W&out_Domain=10YDK-1--------W&securityToken=9539b08a-6f23-413b-94f6-b3b9c26ab3cc&periodStart=202212312300&periodEnd=202306292200'